# Project 3: 2D Simulation

In this project I'll ask you to learn a little but about numpy 2D arrays, then implement a 2D convolution simulation environment, and then implement a variety of models within it.

This project is open-ended, and designed to challenge you.  Along the way you'll have plenty of opportunities to explore and learn concepts that challenge you.

## Implementation Details

You don't *have* to use this notebook to implement your model - you can use an `.ipynb` in Visual Studio, or straight-up python in a `.py` file.  You can use your personal computer or a CS machine (preferred), but one way or another you'll need to ensure that your code runs on a department computer, and that you provide me with all the code needed to run the project.

---

## Part 1: The Simulator

Implement a numpy 2-D array based convolution simulator.  You can use an object oriented model if you would like (and you probably *should* if you have taken CSC-120 or CSC-151).  You simulation should have the following features:

* have variable, rather than hard-coded size (rows,cols).  The size should stay fixed *during* simulation of course.
* a function to initialize the world, i.e. by addding cells at specified points.
* have your own `convolve()` function, which runs 2D convolution on the numpy array.  This function *should not* use the built-in numpy convolve method - I want you to roll your own!  This function should work with variable-sized kernels, not just `3x3` but also `5x5`, `7x7` etc.  (always an odd number).

* have a `step()` function, which modifies the world array using convolution.
* have a `step_n_times` function, which steps a given number of times
* a means of visualizing the world.
* how will you handle boundary conditions?  You might want to learn how to implement keyword arguments (`kwargs`) in python functions.




In [21]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [29]:
rows = random.randint(20,40)
cols = random.randint(20,40)

heatWorld = np.zeros((rows,cols))

def convolve(world, kernel):
    # Flip the kernel
    kernel = np.flipud(np.fliplr(kernel))
    
    # Get dimensions of world and kernel
    world_height, world_width = world.shape
    kernel_height, kernel_width = kernel.shape
    
    # Compute output dimensions
    output_hei = world_height - kernel_height + 1
    output_wid = world_width - kernel_width + 1
    
    #look I did this part for you
    #if you want more precision you can use a different datatype tuan uint8
    output = np.zeros((output_hei, output_wid), np.uint8)
    
    # Perform 2D convolution
    y = output_hei
    x = output_wid   
    output[y, x] = np.sum(world[y:y+kernel_height, x:x+kernel_width] * kernel)
    

def step(world,kernel):
    convolve(world,kernel)

def step_n_times(n,world,kernel):
    for i in range(n):
        step(world,kernel)   






    
   
def plot_iterations(array:np.array, rows:int, cols:int)->None:
    '''
    Downey's code to plot a cellular array in a pretty way
    '''
    #color map
    cmap = plt.get_cmap('Blues')
    #plot using colormap
    plt.imshow(array, cmap=cmap, interpolation='none')


## Part 2: Modeling Heat Diffusion

Use your system above to model heat diffusing across the plane. You can use a simple averaging kernel for this, or something more sophisticated.  Test it with various intensities and locations.

In [30]:

k = np.array([[1/3, 1/3, 1/3],
              [1/3, 1/3, 1/3],
              [1/3, 1/3, 1/3]])

step_n_times(25,heatWorld,k)

plt.plot_iterations(heatWorld,rows,cols)



ValueError: operands could not be broadcast together with shapes (2,2) (3,3) 

## Part 3: Modeling Housing Segregation

(below is excerpted from Downey's "Think Complexity")

In 1969 Thomas Schelling published “Models of Segregation”, which proposed a simple model of racial segregation. You can read it at http://thinkcomplex.com/schell.

The Schelling model of the world is a grid where each cell represents a house. The houses are occupied by two kinds of agents, labeled red and blue, in roughly equal numbers. About 10% of the houses are empty.


At any point in time, an agent might be happy or unhappy, depending on the other agents in the neighborhood, where the “neighborhood" of each house is the set of eight adjacent cells. In one version of the model, agents are happy if they have at least two neighbors like themselves, and unhappy if they have one or zero.

The simulation proceeds by choosing an agent at random and checking to see whether they are happy. If so, nothing happens; if not, the agent chooses one of the unoccupied cells at random and moves.

### Implementation

Modify your 2D cell above as needed to model this system.  You'll need to specify the different types of agent (red vs blue) within each cell.  One way to do this is to have two separate arrays, one for agents of each type. You can use a 3x3 kernel to count how many neighbors of each type are in a neighborhood.

There are other parameters you'll need to establish, for instance the initial density of each population, the size of the model, etc.

You'll probably want a way to visualize the system at various iterations.

### Observation

Run the model for many time steps. What do you observe about this model?  




## Reaction Diffusion Equations

Karl Sims (a hero of mine) has an [interesting explanation](https://karlsims.com/rd.html) of Reaction-Diffusion equations, in which two chemicals circulating in a bath both diffuse through the medium and react with each other.

The below formulas are well explained and annotated on Sim's site.  Don't let the jargon and symbols confuse you.  I'll explain the terms below.

Imagine $A$ and $B$ are chemicals that react with one another.  Specifically, $A$ is converted to $B$ whenever $A$ is near two $B$.

$$A` = A + D_{A}\nabla^{2}A - AB^{2} + f(1-A) \Delta t$$

which can be rewritten as a change equation:

$$ A {+}{=} D_{A}\nabla^{2}A - AB^{2} + f(1-A) $$

Imagine $A$ refers to the percentage of chemical A at any cell in the model.

Where:
*  $D_{A}$ is a constant
*  $\nabla^{2}A$ means 2-D diffusion (using the kernel below)
* $AB^{2}$ means that an $A$ will become a B at any particular cell with a rate determined by the concentration of A and the square of the concentration of B in that cell.   We subtract this term from A (and add it to B)
* $f$ in the equation $f(1-A)$ is the rate at which A is added to the system at any cell.  So if the cell has no A, then $f$ A is added to the cell.

The equation for B is very similar:

$$ B {=} B + D_{B}\nabla^{2}B + AB^{2} - (k+f)B) \Delta t$$

or

 $$B {+}{=}  D_{B}\nabla^{2}B + AB^{2} - (k+f)B)$$

where $k$ is the rate at which B is removed from the system.

### Details

Implement the reaction-diffusion equation described by Sims.  To do this you'll need an array for each chemical.

#### Diffusion

Diffusion  via $\nabla^{2}A$ means each chemical spreads out in the environment. The *diffusion* step is a simple averaging kernel.  I think the following works best:

```
0 1  0
1 -4 1
0 1  0
```

#### Reaction

Reaction via $AB^{2}$ means multiplying each cell in A times the square of the corresponding cell in B.

### Feed and Kill

This means just increasing/or decreasing each cell at a certain rate proportional to its current contents.

### Implementation

Implement this system.  You could start by modifying your code from Part 2.
